<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_evaluators_judegs_and_grader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama langchain_community --quiet
!pip install openai --quiet

host="localhost:11434"
modelid="chevalblanc/gpt-4o-mini"
modelid="deepseek-r1:14b"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [2]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [3]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [4]:
from pydantic import Field, BaseModel

class HallucinationBinaryGrade(BaseModel):
   binary_score: str = Field(description="True if answer is grounded in facts. False if answer is hallucination.")


In [5]:
def call(system_prompt: str, message: str, output_schema: BaseModel | None = None , model: str = modelid, history: list[dict] = [] ) -> str:
  history.append(
          {
              "role": "user",
              "content": message,
          },)
  completion = llm.chat.completions.parse(
      model=modelid,
      messages=[ {"role": "system", "content": system_prompt},]
      +history,
      temperature=0.0,
      max_tokens=200,
      response_format=output_schema
  )

  return completion.choices[0].message.parsed

In [13]:
import json

def grade_hallucination(llm_answer:str, facts: list[str] = []) -> HallucinationBinaryGrade:
  """
  You are a grader assessing wether a LLM generated text is grounded in or supported by a set of retrieved facts.

  Give a binary score "True" or "False". "True" if the text is ground in or supported by the facts. Otherwise return "False"
  """

  prompt = """

  Set of facts:
  - {facts}

  LLM generated text:
  {text}


  """
  facts_as_string = "\n- ".join(facts)
  prompt = prompt.format(facts=facts_as_string, text=llm_answer)

  return call (grade_hallucination.__doc__, prompt, output_schema=HallucinationBinaryGrade)





In [14]:
result = grade_hallucination ("The capitol of France is Madrid", ["Madrid is the capitol of France"])
result

HallucinationBinaryGrade(binary_score='False')